In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [2]:
file_folder = '~/Data/Kaggle/Titanic/'

In [5]:
train_file = file_folder + 'train_processed.csv'

In [6]:
df = pd.read_csv(train_file)

In [7]:
df.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,1,0,0,1,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,1,0,0,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,0,0,0,1,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,1,0,0,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,0,0,1,0,1,0,0,1


In [9]:
Y = df['Survived'].values
X = df.drop(['Survived'], axis=1).values

# Build Random Forest model

In [ ]:
def optimize():
    pass